<a href="https://colab.research.google.com/github/jmsmg/HAI2021/blob/main/HAICon2021(gon).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터셋 준비

## 1-1. Import library

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import time

from keras.models import Model, Sequential, load_model
import keras
from keras import optimizers
from keras.layers import Input,Bidirectional ,LSTM, Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint

from pathlib import Path
from datetime import timedelta
import dateutil
from tqdm.notebook import trange
# from TaPR_pkg import etapr

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1-3. 경로에서 리스트 형태로 데이터 셋 불러오기


In [3]:
TRAIN_DATASET = sorted([x for x in Path("./drive/MyDrive/Datasets/HAICon2021_dataset/train").glob("*.csv")])
TRAIN_DATASET

[PosixPath('drive/MyDrive/Datasets/HAICon2021_dataset/train/train1.csv'),
 PosixPath('drive/MyDrive/Datasets/HAICon2021_dataset/train/train2.csv'),
 PosixPath('drive/MyDrive/Datasets/HAICon2021_dataset/train/train3.csv'),
 PosixPath('drive/MyDrive/Datasets/HAICon2021_dataset/train/train4.csv'),
 PosixPath('drive/MyDrive/Datasets/HAICon2021_dataset/train/train5.csv'),
 PosixPath('drive/MyDrive/Datasets/HAICon2021_dataset/train/train6.csv')]

In [4]:
TEST_DATASET = sorted([x for x in Path("./drive/MyDrive/Datasets/HAICon2021_dataset/test").glob("*.csv")])
TEST_DATASET

[PosixPath('drive/MyDrive/Datasets/HAICon2021_dataset/test/test1.csv'),
 PosixPath('drive/MyDrive/Datasets/HAICon2021_dataset/test/test2.csv'),
 PosixPath('drive/MyDrive/Datasets/HAICon2021_dataset/test/test3.csv')]

In [5]:
VALIDATION_DATASET = sorted([x for x in Path("./drive/MyDrive/Datasets/HAICon2021_dataset/validation").glob("*.csv")])
VALIDATION_DATASET

[PosixPath('drive/MyDrive/Datasets/HAICon2021_dataset/validation/validation.csv')]

## 1-4. 데이터 셋 함수식

In [6]:
# 각 CSV 파일마다 
def dataframe_from_csv(target):
    return pd.read_csv(target).rename(columns=lambda x: x.strip())

# 굳이 칼럼명 공백제거 함수를 만듬(큰그림?) // 이해 안감

###############

# CSV 파일이 여러개의 리스트일때
def dataframe_from_csvs(targets):
    return pd.concat([dataframe_from_csv(x) for x in targets]) 

# pandas의 concat 함수를 사용하여 이어붙이기

# 사실상 함수 안에 함수?

## 1-5. Train dataset 불러오기

In [7]:
# Train dataset load

# train dataset을 리스트로 묶은 형태(앞서 read할때 comprehension으로 list형태로 변수에 담아놨음)

# 모든 Train 데이터 호출됨

TRAIN_DF_RAW = dataframe_from_csvs(TRAIN_DATASET) 
TRAIN_DF_RAW

,timestamp,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,...,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86
0,2021-07-11 10:00:00,-2.2642,0,12.26196,-0.00087,12.01019,0.2254,12.66931,70,1,50,11704,17.61017,28.03686,27170,27.24188,25.42843,0,1,50,27.28778,31.02721,1,267.90363,467.27020,3865,50,14742,0.88504,1,54816,1009.15302,58.21227,5.5352,0,126.92957,2880,26.92264,0,0,...,48.64655,1,0,13752,41.81518,1,0.14465,0.46783,0,276.45758,476.76703,8.90254,49.26452,719.96625,1,923,0,0,16,0.08771,15.07713,9914,1,306.07104,28.20367,15.50464,251.23593,451.06253,26.13983,48.90190,0.92255,30.08042,7.08818,595.06104,276.40338,1,1014.79321,12.0,50,3506
1,2021-07-11 10:00:01,-2.4923,0,12.26196,0.00058,12.56714,-0.0711,12.66931,70,1,50,11847,17.61780,28.03169,27171,27.24188,25.42656,0,1,50,27.28778,31.01196,1,268.95258,466.91238,3898,50,14781,0.88619,1,54816,1011.06647,57.44932,4.7507,0,126.92957,2880,26.92264,0,0,...,50.40893,1,0,13752,41.81518,1,0.14465,0.42892,0,278.91705,476.76703,8.90254,49.26452,720.10663,1,923,0,0,16,0.08771,14.97197,9914,1,303.09189,28.19449,16.20023,250.47298,404.38739,26.13983,48.94870,0.92255,30.08423,7.08818,531.50317,276.18634,1,1014.79321,12.0,50,3493
2,2021-07-11 10:00:02,-2.8460,0,12.26196,-0.00072,14.48975,0.0051,12.66931,70,1,50,11953,17.61780,28.03504,27170,27.24188,25.41979,0,1,50,27.28778,31.01196,1,269.76636,466.91238,3892,50,14831,0.88836,1,54816,1009.79083,58.78445,5.5709,0,126.92957,2880,26.92264,0,0,...,50.60730,1,0,13760,41.81518,1,0.14465,0.42282,0,278.89899,476.76703,8.90254,49.26452,719.96625,1,923,0,0,16,0.08771,14.90129,9914,1,299.11945,28.18530,16.53352,249.90080,382.53925,26.14746,48.89090,0.91873,30.09148,7.08818,451.06253,279.85754,1,1014.79321,12.0,50,3490
3,2021-07-11 10:00:03,-2.1235,0,12.26196,0.00101,15.93170,0.1842,12.66931,70,1,50,12051,17.61780,28.03510,27171,27.24188,25.44255,0,1,50,27.28778,31.01196,1,271.64716,466.35010,3876,50,14917,0.89214,1,54816,1008.77380,59.16594,5.1828,0,126.92957,2880,26.92264,0,0,...,49.14246,1,0,13760,41.81518,1,0.14465,0.43121,0,280.49048,476.76703,8.90254,49.26452,720.00159,1,923,0,0,16,0.08771,14.82246,9914,1,306.07104,28.17612,16.30247,250.47298,396.44250,26.13983,48.95938,0.91644,30.10407,7.08818,404.38739,281.50317,1,1014.79321,12.0,50,3525
4,2021-07-11 10:00:04,-2.9074,0,12.26196,0.00043,16.10718,0.1842,12.66931,70,1,50,12180,17.61780,28.03510,27171,27.24188,25.44007,0,1,50,27.28778,31.02721,1,272.80457,466.60565,3871,50,14997,0.89191,1,54815,1009.79083,58.78445,5.1828,0,126.92957,2880,26.92264,0,0,...,47.18170,1,0,13760,41.81518,1,0.14465,0.45486,0,283.47437,476.76703,8.90254,49.26452,720.00159,1,923,0,0,16,0.08771,14.73087,9914,1,308.05728,28.16693,15.79237,250.28227,427.22849,26.13983,49.02211,0.91797,30.10331,7.08818,382.53925,281.34039,1,1014.79321,12.0,50,3503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259195,2021-08-09 08:59:56,-2.0065,0,12.26196,0.00051,100.28228,0.5977,12.53358,70,1,50,8536,13.45978,28.03433,27213,27.75778,37.62007,0,1,50,27.70997,31.40868,1,318.30511,465.27664,2,50,17293,1.08731,1,54740,991.81366,58.78445,5.7302,0,112.42368,2880,31.38997,0,0,...,12.39929,1,0,0,36.39832,1,64.39886,0.36179,0,307.83423,459.18994,4.67615,49.62311,720.04309,1,905,0,0,13856,0.13800,100.00000,10000,1,308.05728,35.00000,100.00000,242.46214,3175.36084,38.49182,48.07038,1.17355,35.05434,10.00000,3178.48877,370.02679,1,986.05908,12.0,50,136
259196,2021-08-09 08:59:57,-2.2101,0,12.26196,-0.00029,100.28228,0.5509,12.53358,70,1,50,8519,13.45978,28.03521

## 1-6. column 제거, 변수화

In [8]:
# column명 변수화

TIMESTAMP_FIELD = 'timestamp'
IDSTAMP_FIELD = 'id'
ATTACK_FIELD = 'attack'


VALID_COLUMNS_IN_TRAIN_DATASET = TRAIN_DF_RAW.columns.drop([TIMESTAMP_FIELD])
VALID_COLUMNS_IN_TRAIN_DATASET

Index(['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10',
       'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30',
       'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40',
       'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50',
       'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60',
       'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70',
       'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80',
       'C81', 'C82', 'C83', 'C84', 'C85', 'C86'],
      dtype='object')

# 2. Data Preprocessing

## 2-1. feature scaling

### feature scaling

1. standardization
  > 정규화는 회귀, SVM(서포트 벡터 머신), NN(신경망)에서는 효과적이다
2. nomalization 

  > numerical column이기 때문에

### nomalization 종류

1. min-max scaling

2. Z-scroe Nomalization

- Min-Max를 선택한 이유는 편향되지않아서 선택
- Z-score는 분포가 집중되어 outlier 잡아내기는 쉽지만 분포를 고르게 하기에는 적합하지않음


In [9]:
TAG_MIN = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].min()
TAG_MAX = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].max()

# 원본 train data 에서 유효칼럼들의 최소값 최대값 추출

## 2-2. min-max normalization

- 최대값과 최소값이 같으면 값이 전혀 바뀌지 않음

> 여기선 이런 필드를 0으로 두었음(0~1 사이 다른 수를 대입하면 결과가 어떻게 나올까?)

In [10]:
def normalize(df):
    ndf = df.copy() # deep copy
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]: # 최소값과 최대값이 같을때 
            ndf[c] = df[c] - TAG_MIN[c] # 전체 데이터 프레임에서 최소값을 뺀다.
        else: 
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c]) # (전체 - 최소값) / (최대값 - 최소값)
    return ndf

## 2-3. Noise 처리

### 시계열 데이터 분석 방법
  - 회귀분석
  - Box-Jenkins 방법
  - ⭐**지수평활법**(exponential weighted function)
  - 시계열 분해법

---
### 지수평활법
  - 센서에서 발생하는 noise를 smoothing 
  - exponential weighted function(지수 평활법)
  - x값이 음수인 지수함수만큼 가중치가 줄어듬(과거의 경향성 // 지수적 감쇠)
  - $\frac{1}{1-\beta }$식으로 나온다
    > β​​값이 0.98이면 50이 나와 50 이후는 영향력이 없어짐

https://m.blog.naver.com/nilsine11202/221903150950

<details>
<summary>지수 평활법 특징</summary>  

1. 단기간에 발생하는 불규칙변동을 평활하는 방법

2. 자료의 수가 많고 안정된 패턴을 보이는 경우일수록 예측 품질이 높음

3. 지수평활법에서 가중치의 역할을 하는 것은 지수평활계수(α)이며, 불규칙변동이 큰 시계열의 경우 지수평활계수는 작은 값을, 불규칙변   동이 작은 시계열의 경우, 큰 값의 지수평활계수를 적용함(generally, α is between 0.05 and 0.3)

4. 지수평활계수는 예측오차(실제 관측치와 예측치 사이의 잔차제곱합)를 비교하여 예측오차가 가장 작은 값을 선택하는 것이 바람직 함

5. 지수평활계수는 과거로 갈수록 지수적으로 감소함

6. 지수평활법은 불규칙변동의 영향을 제거하는 효과가 있으며, 중기 예측 이상에 주로 사용됨(단, 단순지수 평활법의 경우, 장기추세나 계   절변동이 포함된 시계열의 예측에는 적합하지 않음)


</details>

In [11]:
TRAIN_DF = normalize(TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET]).ewm(alpha=0.9).mean() # 데이터 정규화, 지수 가중 이동 평균
TRAIN_DF


,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,...,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86
0,0.433107,0.0,0.056338,0.501686,0.116706,0.447955,0.816071,0.0,0.0,0.0,0.390156,0.501876,0.520480,0.449040,0.252360,0.193673,0.0,0.0,0.0,0.305026,0.452537,0.0,0.133292,0.578120,0.626384,0.0,0.135313,0.132221,0.0,0.976296,0.513118,0.259863,0.369939,0.0,0.900937,0.0,0.209725,0.0,0.0,0.029799,...,0.424797,1.0,0.0,0.994220,0.471624,0.0,0.000575,0.130348,0.0,0.172978,0.798452,0.178051,0.241962,0.465834,1.0,0.909722,0.0,0.0,0.007323,0.22613,0.150771,0.207547,0.0,0.321628,0.805819,0.155046,0.443864,0.138395,0.191217,0.246855,0.240561,0.194185,0.708818,0.183693,0.169066,0.0,0.538147,0.0,0.0,0.276078
1,0.395098,0.0,0.056338,0.517560,0.121738,0.397053,0.816071,0.0,0.0,0.0,0.397402,0.502077,0.403097,0.450383,0.252360,0.193649,0.0,0.0,0.0,0.305026,0.450531,0.0,0.136943,0.576120,0.631199,0.0,0.137850,0.133443,0.0,0.976296,0.519266,0.256744,0.245047,0.0,0.900937,0.0,0.209725,0.0,0.0,0.028294,...,0.443542,1.0,0.0,0.994220,0.471624,0.0,0.000575,0.101569,0.0,0.181368,0.798452,0.178051,0.241962,0.596204,1.0,0.909722,0.0,0.0,0.007323,0.22613,0.149815,0.207547,0.0,0.318762,0.805581,0.161370,0.439117,0.125047,0.191217,0.248731,0.240561,0.194535,0.708818,0.165517,0.168332,0.0,0.538147,0.0,0.0,0.275176
2,0.333267,0.0,0.056338,0.504886,0.139408,0.405431,0.816071,0.0,0.0,0.0,0.403379,0.502095,0.467897,0.449173,0.252360,0.193558,0.0,0.0,0.0,0.305026,0.450351,0.0,0.140079,0.575940,0.630765,0.0,0.141302,0.135839,0.0,0.976296,0.515758,0.261871,0.363195,0.0,0.900937,0.0,0.209725,0.0,0.0,0.029108,...,0.447321,1.0,0.0,0.994740,0.471624,0.0,0.000575,0.094505,0.0,0.182062,0.798452,0.178051,0.241962,0.478753,1.0,0.909722,0.0,0.0,0.007323,0.22613,0.149092,0.207547,0.0,0.314716,0.805323,0.164942,0.435161,0.117652,0.191315,0.246604,0.238319,0.195226,0.708818,0.141083,0.180570,0.0,0.538147,0.0,0.0,0.274889
3,0.446345,0.0,0.056338,0.522383,0.154058,0.436703,0.816071,0.0,0.0,0.0,0.408887,0.502096,0.475662,0.450383,0.252360,0.193846,0.0,0.0,0.0,0.305026,0.450333,0.0,0.146871,0.572809,0.628411,0.0,0.147183,0.140053,0.0,0.976296,0.512176,0.263923,0.313719,0.0,0.900937,0.0,0.209725,0.0,0.0,0.031288,...,0.432270,1.0,0.0,0.994792,0.471624,0.0,0.000575,0.099950,0.0,0.187506,0.798452,0.178051,0.241962,0.499620,1.0,0.909722,0.0,0.0,0.007323,0.22613,0.148311,0.207547,0.0,0.320938,0.805061,0.163216,0.438294,0.120857,0.191227,0.249111,0.236754,0.196440,0.708818,0.125447,0.187292,0.0,0.538147,0.0,0.0,0.277264
4,0.328324,0.0,0.056338,0.517845,0.157091,0.439827,0.816071,0.0,0.0,0.0,0.415910,0.502097,0.476437,0.450504,0.252360,0.193843,0.0,0.0,0.0,0.305026,0.452317,0.0,0.151538,0.573911,0.627454,0.0,0.152922,0.140232,0.0,0.974963,0.515053,0.262584,0.308776,0.0,0.900937,0.0,0.209725,0.0,0.0,0.033876,...,0.410120,1.0,0.0,0.994797,0.471624,0.0,0.000575,0.117812,0.0,0.198128,0.798452,0.178051,0.241962,0.501704,1.0,0.909722,0.0,0.0,0.007323,0.22613,0.147409,0.207547,0.0,0.323451,0.804799,0.158453,0.437432,0.129893,0.191218,0.251851,0.237495,0.196492,0.708818,0.117700,0.187419,0.0,0.538147,0.0,0.0,0.275990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259195,0.485034,0.0,0.056340,0.515751,0.994085,0.520004,0.640361,0.0,0.0,0.0,0.213613,0.381810,0.459083,0.511453,0.371984,0.370597,0.0,0.0,0.0,0.752517,0.507725,0.0,0.326726,0.565976,0.006420,0.0,0.318660,0.368856,0.0,0.863704,0.452023,0.262573,0.404346,0.0,0.413326,0.0,0.748145,0.0,0.0,0.976893,...,0.000714,1.0,0.0,0.000578,0.124171,0.0,0.68258

## 2-4. Boundary Check(경계검사)

- min_max scaling으로 0~1값으로 넣었는데 밖에 있거나, nan값이 있는지 확인
  > 1-4. 데이터셋 함수식에서 이미 최대 최소 같을때 X데이터 - 최소로 지정해서 그럴리 없을듯 고로 벗어나면 attack
   
  > x값 - 최소값 / 최대값 - 최소값


In [12]:
def boundary_check(df):
    x = np.array(df, dtype=np.float32)
    return np.any(x > 1.0), np.any(x < 0), np.any(np.isnan(x))

    # np.any 함수는 조건을 만족하는 1개 이상의 element가 있는지 찾아줌

In [13]:
boundary_check(TRAIN_DF)

# (1보다 큰값 유무, 0보다 작은값 유무, 결측치 유무)

(False, False, False)

## 2-5. 데이터셋 구성

- dataset에 답 표기가 없으므로 unsupervised learning으로 모델을 진행하여야 함

  - timestep을 입력값으로 받는 DeepLearning 모델 RNN(LSTM,GRU,seq2seq)을 선택
    
    - vanila RNN모델은 gradient vanishing, gradient exploding 문제(이해 아직 못함) // overfitting?

    - sliding window algorithm 이해 필요

      > 영상 링크 [sliding window algorithm](https://youtu.be/uH9VJRIpIDY)  
      > 영상 링크 [LSTM](https://youtu.be/bX6GLbpw-A4)

In [14]:
WINDOW_GIVEN = 59 # 윈도우 앞부분 59초에 해당하는 값
WINDOW_SIZE= 60 # 윈도우 가장 마지막 60초에 해당하는 값

def HaiDataset(timestamps, df, stride=1, attacks=None):
    ts= np.array(timestamps)  # ts = timestamp는 데이터 Nomalization을 위해서 잠깐 삭제했던거??(?)
    tag_values=np.array(df,dtype=np.float32) # tag_values를 np.array형태로 넣었고, float32는 용량문제
    valid_idxs=[]

# 여기부턴 작업 진행상황 확인을 위한 문구
    for L in trange(len(ts)-WINDOW_SIZE+1): # trange는 tqdm 라이브러리에서 가져옴
        R = L + WINDOW_SIZE - 1
        if dateutil.parser.parse(ts[R]) - dateutil.parser.parse(ts[L]) == timedelta(seconds=WINDOW_SIZE - 1):
            valid_idxs.append(L)
    valid_idxs=np.array(valid_idxs, dtype=np.int32)[::stride]
    n_idxs = len(valid_idxs)
    print("# of valid windows:", n_idxs)
# 진행상황 확인은 여기까지

    if attacks is not None:
        attacks = np.array(attacks, dtype=np.float32)
        with_attack = True
    else:
        with_attack = False
        
    
    timestamp, X, y, att = list(), list(), list(), list()
    
    if with_attack:
        for i in valid_idxs:
            last=i+WINDOW_SIZE-1
            seq_time, seq_x, seq_y, seq_attack = ts[last], tag_values[i:i+WINDOW_GIVEN], tag_values[last], attacks[last]
            timestamp.append(seq_time)
            X.append(seq_x)
            y.append(seq_y)
            att.append(seq_attack)
            
        return np.array(timestamp), np.array(X), np.array(y), np.array(att)
    else:
        for i in valid_idxs:
            last=i+WINDOW_SIZE-1
            seq_time, seq_x, seq_y = ts[last], tag_values[i:i+WINDOW_GIVEN], tag_values[last]
            timestamp.append(seq_time)
            X.append(seq_x)
            y.append(seq_y)
        return np.array(timestamp), np.array(X), np.array(y)

In [ ]:
# ts, X_train, y_train = HaiDataset(TRAIN_DF_RAW[TIMESTAMP_FIELD], TRAIN_DF, stride=1)

# haiDataset(timestamp, 전처리 완료 된 Train data, 슬라이딩 할때의 크기(stride)
# attack은 none처리

  0%|          | 0/1004343 [00:00<?, ?it/s]

# of valid windows: 1004048


In [18]:
## Validation dataset load

VALIDATION_DF_RAW = dataframe_from_csvs(VALIDATION_DATASET)
VALIDATION_DF_RAW

,timestamp,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,...,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,attack
0,2021-07-10 00:00:01,-2.0395,0,11.78894,0.00058,16.32080,0.4010,12.903438,70,1,50,10289,11.68213,28.03510,27181,27.84305,29.50472,0,1,50,27.796206,31.04248,1,334.58112,437.98016,3,50,18131,1.04366,1,54830,1032.43347,57.25861,4.9870,0,129.60573,2880,27.736847,0,0,...,1,0,0,45.33997,1,71.77734,0.44952,0,292.86029,431.328095,20.989594,51.26343,720.02826,1,927,0,0,15184,0.059445,14.91310,10000,1,300.11276,35.00000,15.09755,264.01520,438.15259,30.90057,50.63332,1.22543,31.52100,0.716042,386.51172,356.64423,1,1034.712769,12.0,50,161,0
1,2021-07-10 00:00:02,-2.1750,0,11.78894,-0.00282,15.95459,0.4187,12.903438,70,1,50,10266,11.68213,28.03419,27174,27.84305,29.41619,0,1,50,27.796206,31.04248,1,332.97162,437.62238,2,50,18111,1.04217,1,54829,1032.43347,57.06784,5.3008,0,129.60573,2880,27.736847,0,0,...,1,0,0,45.33997,1,74.29114,0.44723,0,295.82605,431.328095,20.989594,51.26343,719.85382,1,926,0,0,14224,0.059445,14.95540,9999,1,298.12650,35.00000,14.71477,264.01520,465.95908,30.88531,50.64244,1.22543,31.51756,0.716042,411.33905,352.08698,1,1034.712769,12.0,50,155,0
2,2021-07-10 00:00:03,-2.5854,0,11.78131,-0.00398,14.79492,0.0691,12.903438,70,1,50,10238,11.68213,28.03429,27172,27.84305,29.31749,0,1,50,27.796206,31.04248,1,332.15784,437.11121,2,50,18100,1.03920,1,54828,1031.79565,56.68636,4.5175,0,129.60573,2880,27.736847,0,0,...,1,0,0,45.33997,1,70.54761,0.43884,0,300.02173,431.328095,20.989594,51.26343,719.92407,1,927,0,0,12368,0.059445,14.99200,9999,1,297.13321,35.00000,14.30731,263.63373,452.05582,30.58014,50.65156,1.21475,31.50269,0.716042,438.15259,347.63818,1,1034.712769,12.0,50,149,0
3,2021-07-10 00:00:04,-2.4722,0,11.78894,-0.00152,14.72626,0.3306,12.903438,70,1,50,10215,11.68213,28.03686,27168,27.84305,29.26632,0,1,50,27.796206,31.04248,1,330.96429,437.16232,2,50,18000,1.03222,1,54829,1033.07117,56.68636,5.4640,0,129.60573,2880,27.736847,0,0,...,1,0,8,45.33997,1,59.44373,0.42435,0,302.10144,431.328095,20.989594,51.26343,720.02954,1,924,0,0,11888,0.059445,15.01359,9999,1,295.14697,35.00000,14.54943,264.39661,434.18011,30.06897,50.68747,1.20255,31.47942,0.716042,465.95908,340.33203,1,1034.712769,12.0,50,148,0
4,2021-07-10 00:00:05,-2.2319,0,11.78131,0.00072,14.78729,0.2150,12.903438,70,1,50,10085,11.68213,28.02990,27172,27.84305,29.20597,0,1,50,27.796206,31.04248,1,329.02917,437.11121,2,50,17852,1.02947,1,54830,1031.47668,55.73270,4.5850,0,129.60573,2880,27.736847,0,0,...,1,0,688,45.33997,1,57.27356,0.40451,0,305.03113,431.328095,20.989594,51.14899,720.02905,1,927,0,0,10688,0.059445,15.04767,9999,1,295.14697,35.00000,15.24461,263.44296,411.33905,30.05371,50.64556,1.20483,31.47942,0.716042,452.05582,335.17798,1,1034.712769,12.0,50,148,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86395,2021-07-10 23:59:56,-2.8150,0,12.26196,0.00130,15.53497,-0.0507,12.669310,70,1,50,6494,14.42108,28.02736,27265,27.86172,37.79226,0,1,50,27.796206,31.98853,1,354.96240,455.76889,3,50,19244,1.24152,1,54727,1021.90936,70.03782,5.2654,0,109.91794,2880,27.804180,0,0,...,1,0,8,39.84681,1,0.56064,1.38260,0,347.45734,451.941960,20.989594,50.21057,720.00098,1,905,0,0,16,0.193330,35.65305,10022,1,360.69107,25.53188,15.29280,257.72092,1179.76929,38.01117,49.54745,1.25824,31.79814,0.716042,1193.48230,347.58386,1,1018.394040,12.0,50,7,0
86396,2021-07-10 23:59:57,-2.6272,0,12.26196,0.00109,15.53497,0.1829,12.669310,70,1,50,6496,14.41345,28.03598,27262,27.86

In [19]:
VALIDATION_DF = normalize(VALIDATION_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET])

In [20]:
boundary_check(VALIDATION_DF)

(True, True, False)

In [21]:
ts, X_valid, y_valid, attack = HaiDataset(VALIDATION_DF_RAW[TIMESTAMP_FIELD], VALIDATION_DF, attacks=VALIDATION_DF_RAW[ATTACK_FIELD])

  0%|          | 0/86341 [00:00<?, ?it/s]

# of valid windows: 86341


In [22]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_LSTM_model_60seq_TEST.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [23]:
## LSTM Model define

n_features = TRAIN_DF.shape[1]

inputs=Input(shape=(WINDOW_GIVEN , n_features))
first=Bidirectional(LSTM(100, return_sequences=True))(inputs)
second=Bidirectional(LSTM(100, return_sequences=True))(first)
thrid=Bidirectional(LSTM(100))(second)

lstm_out=Dense(n_features)(thrid)

aux_input = Input(shape=(n_features,), name='aux_input')

outputs = keras.layers.add([lstm_out, aux_input])

model_60seq=Model(inputs=[inputs, aux_input], outputs=outputs)

model_60seq.compile(loss='mean_squared_error', optimizer='Adam')

model_60seq.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 59, 86)]     0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 59, 200)      149600      input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 59, 200)      240800      bidirectional[0][0]              
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 200)          240800      bidirectional_1[0][0]            
______________________________________________________________________________________________

In [28]:
aux_train=[]
for i in range(len(X_train)):
    aux_train.append(X_train[i][0])
aux_train=np.array(aux_train)

KeyError: ignored

In [25]:
aux_valid=[]
for i in range(len(X_valid)):
    aux_valid.append(X_valid[i][0])
aux_valid=np.array(aux_valid)

In [ ]:
hist=model_60seq.fit([X_train, aux_train], y_train, batch_size=512, epochs=32, shuffle=True, 
               callbacks=[es, mc], validation_data=([X_valid,aux_valid],y_valid))

# 4. Skip learning

In [ ]:
## model load

model_60seq = load_model('best_LSTM_model_60seq.h5')

In [ ]:
## Validation set Prediction


y_pred=model_60seq.predict([X_valid,aux_valid])

In [ ]:
tmp=[]
for i in range(len(y_pred)):
    tmp.append(abs(y_valid[i]-y_pred[i]))

In [ ]:
ANOMALY_SCORE=np.mean(tmp,axis=1)

In [ ]:
def put_labels(distance, threshold):
    xs = np.zeros_like(distance)
    xs[distance > threshold] = 1
    return xs

In [ ]:
THRESHOLD = 0.027

LABELS = put_labels(ANOMALY_SCORE, THRESHOLD)
LABELS, LABELS.shape

In [ ]:
ATTACK_LABELS = put_labels(np.array(VALIDATION_DF_RAW[ATTACK_FIELD]), threshold=0.5)
ATTACK_LABELS, ATTACK_LABELS.shape

In [ ]:
def fill_blank(check_ts, labels, total_ts):
    def ts_generator():
        for t in total_ts:
            yield dateutil.parser.parse(t)

    def label_generator():
        for t, label in zip(check_ts, labels):
            yield dateutil.parser.parse(t), label

    g_ts = ts_generator()
    g_label = label_generator()
    final_labels = []

    try:
        current = next(g_ts)
        ts_label, label = next(g_label)
        while True:
            if current > ts_label:
                ts_label, label = next(g_label)
                continue
            elif current < ts_label:
                final_labels.append(0)
                current = next(g_ts)
                continue
            final_labels.append(label)
            current = next(g_ts)
            ts_label, label = next(g_label)
    except StopIteration:
        return np.array(final_labels, dtype=np.int8)

In [ ]:
%%time
FINAL_LABELS = fill_blank(ts, LABELS, np.array(VALIDATION_DF_RAW[TIMESTAMP_FIELD]))
FINAL_LABELS.shape

In [ ]:
ATTACK_LABELS.shape[0] == FINAL_LABELS.shape[0]

In [ ]:
TaPR = etapr.evaluate(anomalies=ATTACK_LABELS, predictions=FINAL_LABELS)
print(f"F1: {TaPR['f1']:.3f} (TaP: {TaPR['TaP']:.3f}, TaR: {TaPR['TaR']:.3f})")
print(f"# of detected anomalies: {len(TaPR['Detected_Anomalies'])}")
print(f"Detected anomalies: {TaPR['Detected_Anomalies']}")

# 5. Moving Average

In [ ]:
## MOVING AVERAGE

seq60_10mean=[]
for idx in range(len(ANOMALY_SCORE)):
    if idx >= 10:
        seq60_10mean.append((ANOMALY_SCORE[idx-10:idx].mean()+ANOMALY_SCORE[idx])/2)
    else:
        seq60_10mean.append(ANOMALY_SCORE[idx])

seq60_10mean=np.array(seq60_10mean)
print(seq60_10mean.shape)

In [ ]:
THRESHOLD = 0.019

LABELS = put_labels(seq60_10mean, THRESHOLD)
LABELS, LABELS.shape

In [ ]:
%%time
FINAL_LABELS = fill_blank(ts, LABELS, np.array(VALIDATION_DF_RAW[TIMESTAMP_FIELD]))
FINAL_LABELS.shape

In [ ]:
ATTACK_LABELS.shape[0] == FINAL_LABELS.shape[0]

In [ ]:
TaPR = etapr.evaluate(anomalies=ATTACK_LABELS, predictions=FINAL_LABELS)
print(f"F1: {TaPR['f1']:.3f} (TaP: {TaPR['TaP']:.3f}, TaR: {TaPR['TaR']:.3f})")
print(f"# of detected anomalies: {len(TaPR['Detected_Anomalies'])}")
print(f"Detected anomalies: {TaPR['Detected_Anomalies']}")

In [ ]:
############### Test data Start

TEST_DF_RAW = dataframe_from_csvs(TEST_DATASET)
TEST_DF_RAW

In [ ]:
TEST_DF = normalize(TEST_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET]).ewm(alpha=0.9).mean()
TEST_DF

In [ ]:
boundary_check(TEST_DF)

In [ ]:
CHECK_TS, X_test, y_test = HaiDataset(TEST_DF_RAW[TIMESTAMP_FIELD], TEST_DF, attacks=None)

In [ ]:
aux_test=[]
for i in range(len(X_test)):
    aux_test.append(X_test[i][0])
aux_test=np.array(aux_test)

In [ ]:
## Model Prediction


y_pred=model_60seq.predict([X_test,aux_test])

In [ ]:
tmp=[]
for i in range(len(y_test)):
    tmp.append(abs(y_test[i]-y_pred[i]))

In [ ]:
ANOMALY_SCORE=np.mean(tmp,axis=1)

In [ ]:
# Moving Average

seq60_10mean=[]
for idx in range(len(ANOMALY_SCORE)):
    if idx >= 10:
        seq60_10mean.append((ANOMALY_SCORE[idx-10:idx].mean()+ANOMALY_SCORE[idx])/2)
    else:
        seq60_10mean.append(ANOMALY_SCORE[idx])

seq60_10mean=np.array(seq60_10mean)
print(seq60_10mean.shape)

In [ ]:
### Threshold Setting

THRESHOLD=0.019

LABELS_60seq = put_labels(seq60_10mean, THRESHOLD)
LABELS_60seq, LABELS_60seq.shape

In [ ]:
submission = pd.read_csv('./HAI 2.0/sample_submission.csv')
submission.index = submission['time']
submission.loc[CHECK_TS,'attack'] = LABELS_60seq

submission

In [ ]:
submission.to_csv('SILab_Moving_Average.csv', index=False)

# 6. Gray Aare Smoothing

In [ ]:
submission = pd.read_csv('./HAI 2.0/sample_submission.csv')
submission.index = submission['time']
submission['attack_1']=0
submission['anomaly_score']=0

submission.loc[CHECK_TS,'anomaly_score'] = seq60_10mean
submission.loc[CHECK_TS,'attack_1'] = LABELS_60seq

LABELS_60seq=submission['attack_1']
seq60_10mean=submission['anomaly_score']

In [ ]:
def Gray_Area(attacks):
    start = []  # start point
    finish = []  # finish point
    c = []  # count
    com = 0
    count = 0
    for i in range(1, len(attacks)):
        if attacks[i - 1] != attacks[i]:
            if com == 0:
                start.append(i)
                count = count + 1
                com = 1
            elif com == 1:
                finish.append(i - 1)
                c.append(count)
                count = 0
                start.append(i)
                count = count + 1
        else:
            count = count + 1

    finish.append(len(attacks) - 1)
    c.append(finish[len(finish) - 1] - start[len(start) - 1] + 1)

    for i in range(0, len(start)):
        if c[i] < 10:
            s = start[i]
            f = finish[i] + 1
            g1 = [1 for i in range(c[i])] # Temp Attack list
            g0 = [0 for i in range(c[i])]  # Temp Normal List
            if attacks[start[i] - 1] == 1:
                attacks[s:f] = g1  # change to attack
            else:
                attacks[s:f] = g0  # change to normal

    return attacks

In [ ]:
gray_LABELS_60seq=Gray_Area(LABELS_60seq)

In [ ]:
submission = pd.read_csv('./HAI 2.0/sample_submission.csv')
submission.index = submission['time']

submission['attack']=gray_LABELS_60seq

submission

In [ ]:
submission.to_csv('SILab_Gray_Area_Smoothing.csv', index=False)

# 7. Attack Point and Finish Point Policy


# 8. 10Seq Training Model 

In [ ]:
WINDOW_GIVEN = 9
WINDOW_SIZE= 10

In [ ]:
ts, X_train, y_train = HaiDataset(TRAIN_DF_RAW[TIMESTAMP_FIELD], TRAIN_DF, stride=1)

In [ ]:
ts, X_valid, y_valid, attack = HaiDataset(VALIDATION_DF_RAW[TIMESTAMP_FIELD], 
                                          VALIDATION_DF, attacks=VALIDATION_DF_RAW[ATTACK_FIELD])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_LSTM_model_10seq_TEST.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [ ]:
## LSTM Model define

n_features = TRAIN_DF.shape[1]

inputs=Input(shape=(WINDOW_GIVEN , n_features))
first=Bidirectional(LSTM(100, return_sequences=True))(inputs)
second=Bidirectional(LSTM(100, return_sequences=True))(first)
thrid=Bidirectional(LSTM(100))(second)

lstm_out=Dense(n_features)(thrid)

aux_input = Input(shape=(n_features,), name='aux_input')

outputs = keras.layers.add([lstm_out, aux_input])

model_10seq=Model(inputs=[inputs, aux_input], outputs=outputs)

model_10seq.compile(loss='mean_squared_error', optimizer='Adam')

model_10seq.summary()

In [ ]:
aux_train=[]
for i in range(len(X_train)):
    aux_train.append(X_train[i][0])
aux_train=np.array(aux_train)

In [ ]:
aux_valid=[]
for i in range(len(X_valid)):
    aux_valid.append(X_valid[i][0])
aux_valid=np.array(aux_valid)

In [ ]:
hist=model_10seq.fit([X_train, aux_train], y_train, batch_size=512, epochs=32, shuffle=True, 
               callbacks=[es, mc], validation_data=([X_valid,aux_valid],y_valid))

# Skip learning

In [ ]:
## model load

model_10seq = load_model('best_LSTM_model_10seq.h5')

In [ ]:
CHECK_TS, X_test, y_test = HaiDataset(TEST_DF_RAW[TIMESTAMP_FIELD], TEST_DF, attacks=None)

In [ ]:
aux_test=[]
for i in range(len(X_test)):
    aux_test.append(X_test[i][0])
aux_test=np.array(aux_test)

In [ ]:
## Model Prediction

y_pred=model_10seq.predict([X_test,aux_test])

In [ ]:
tmp=[]
for i in range(len(y_test)):
    tmp.append(abs(y_test[i]-y_pred[i]))

In [ ]:
ANOMALY_SCORE=np.mean(tmp,axis=1)

In [ ]:
# Moving Average

seq10_10mean=[]
for idx in range(len(ANOMALY_SCORE)):
    if idx >= 10:
        seq10_10mean.append((ANOMALY_SCORE[idx-10:idx].mean()+ANOMALY_SCORE[idx])/2)
    else:
        seq10_10mean.append(ANOMALY_SCORE[idx])

seq10_10mean=np.array(seq10_10mean)
print(seq10_10mean.shape)

In [ ]:
### Threshold Setting

THRESHOLD=0.008

LABELS_10seq = put_labels(seq10_10mean, THRESHOLD)

LABELS_10seq, LABELS_10seq.shape

In [ ]:
submission = pd.read_csv('./HAI 2.0/sample_submission.csv')
submission.index = submission['time']
submission['attack_1']=0

submission.loc[CHECK_TS,'attack_1'] = LABELS_10seq

LABELS_10seq=submission['attack_1']

In [ ]:
gray_LABELS_10seq=Gray_Area(LABELS_10seq)

In [ ]:
Queue=[0 for i in range(59)]

Label=[]

for i in range(len(gray_LABELS_60seq)):
    Queue.append(gray_LABELS_60seq[i])
    N=Queue.count(1)
    
    if N>=60:
        if seq60_10mean[i-100:i].max() > 0.1:
            Label.append(gray_LABELS_10seq[i])
        else:
            Label.append(gray_LABELS_60seq[i])
        
    elif N>=1:
        Label.append(gray_LABELS_60seq[i])
    
    else:
        Label.append(gray_LABELS_60seq[i])
    
    del Queue[0]

Label=np.array(Label)

In [ ]:
submission = pd.read_csv('./HAI 2.0/sample_submission.csv')
submission.index = submission['time']

submission['attack']=Label

submission

In [ ]:
submission.to_csv('200929_final_submission.csv', index=False)